# CAPSTONE PROJECT 📊👩🏽‍💻:
*Unicorn E-Commerce Company* |
**SQL Project** |
*Brittney Kakie* 👩🏽‍🦱 

In [2]:
import pandasql as sqldf
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [3]:
conn = sqlite3.connect('Unicorn-1.db')
c = conn.cursor()

***pop_profits_sale VIEW:***

order_id | state | city | product_name | category | qty_ordered | sales | profits | profit_margin | region | day | month | year

In [ ]:
pd.read_sql('''
CREATE VIEW pop_profits_sale AS 
SELECT
  o.order_id AS order_id,
  o.shipping_state AS state,
  o.shipping_city AS city,
  p.product_name AS product_name, 
  p.product_category AS category,
  od.quantity AS qty_ordered,
  od.order_sales AS sales,
  od.order_profits AS profits,
  od.order_profit_ratio AS profit_margin,
  o.shipping_region AS region,
  Substr(order_date, Instr(order_date, "/") + 1,
              Length(order_date) - 5 - Instr(order_date, "/")) AS day,
       Substr(order_date, 0, Instr(order_date, "/"))           month,
       Substr(order_date, -4)                                  year
FROM 
  orders AS o 
  JOIN order_details AS od ON o.order_id = od.order_id 
  JOIN product AS p ON od.product_id = p.product_id
  ''', conn)
  

## 1. How many Customers do we have in the data?

-- num_customers

In [4]:
pd.read_sql('''
SELECT COUNT (*) AS num_customers FROM customers
''', conn).head()

,num_customers
0,795


## 2. What was the city with the most profit for the company in 2015 and how much was it? 

-- city| state | total_profits

In [176]:
pd.read_sql('''
SELECT 
  city, 
  state, 
  SUM(profits) AS total_profits
FROM 
  pop_profits_sale1
WHERE 
  year = '2015'
GROUP BY
  1
ORDER BY
  3 DESC
''', conn).head()

,city,state,total_profits
0,New York City,New York,14753
1,Seattle,Washington,5071
2,Minneapolis,Minnesota,4695
3,San Francisco,California,4290
4,Los Angeles,California,4092


## 3. How many different cities do we have in the data?

-- distinct_city_ct

In [25]:
pd.read_sql('''
SELECT COUNT (DISTINCT(shipping_city)) AS distinct_city_ct
FROM orders
''', conn)

,distinct_city_ct
0,531


## 4. Show the total spent by customers from low to high.

-- customer_id | name| total_spent

In [5]:
pd.read_sql('''
SELECT 
  c.customer_id, 
  c.customer_name AS name, 
  SUM(od.order_sales) AS total_spent 
FROM 
  customers AS c 
  JOIN orders AS o ON c.customer_id = o.customer_id 
  JOIN order_details AS od ON o.order_id = od.order_id 
GROUP BY 
  2 
ORDER BY 
  3
''', conn)

,customer_id,name,total_spent
0,456,Lela Donovan,5
1,738,Thais Sissman,5
2,546,Mitch Gastineau,16
3,124,Carl Jackson,17
4,657,Roy Skaria,22
...,...,...,...
788,758,Tom Ashbrook,14596
789,623,Raymond Buch,15117
790,388,John Lee,16744
791,731,Tamara Chand,19050


## 5. What is the most profitable City in the State of Tennessee?

-- TN_city | total_profits

In [74]:
pd.read_sql('''
SELECT 
  o.shipping_city AS TN_city, 
  SUM(od.order_profits) AS total_profits 
FROM 
  orders AS o 
  JOIN order_details AS od ON o.order_id = od.order_id 
WHERE 
  o.shipping_state = "Tennessee" 
GROUP BY 
  1 
ORDER BY 
  2 DESC
LIMIT
  1
''', conn)

,TN_city,total_profits
0,Lebanon,83


## 6. What’s the average annual profit for that city across all years in that city?

-- TN_city | avg_profit

In [82]:
pd.read_sql('''
SELECT 
  city AS TN_city, 
  AVG(profits) AS avg_profit
FROM 
  pop_profits_sale1 
WHERE 
  1 = 1 
  AND state = "Tennessee" 
  AND city = "Lebanon" 
ORDER BY 
  2 DESC
''', conn)

,TN_city,avg_profit
0,Lebanon,27.666667


## 7. What is the distribution of customer types in the data?

-- customer_type | num_customers

In [4]:
pd.read_sql('''
SELECT 
    Distinct customer_segment AS customer_type
    ,COUNT(customer_id) AS num_customers
FROM customers
GROUP BY 1
''', conn)

,customer_type,num_customers
0,Consumer,410
1,Corporate,237
2,Home Office,148


## 8. What’s the most profitable product category on average in Iowa across all years?

***pop_profits_sale VIEW:***

*order_id | state | city | product_name | category | qty_ordered | sales | profits | profit_margin | region | day | month | year*

-- category | total_profits

In [10]:
pd.read_sql('''
SELECT 
  category,
  profits AS total_profits
FROM 
  pop_profits_sale
WHERE 
  state = "Iowa" 
GROUP BY 
  1
ORDER BY
  2 DESC
''', conn)

,category,total_profits
0,Furniture,394
1,Technology,109
2,Office Supplies,13


## 9. What is the most popular product in that category across all states in 2016?

***pop_profits_sale VIEW:***

*order_id | state | city | product_name | category | qty_ordered | sales | profits | profit_margin | region | day | month | year*

-- product_name | category | total_ordered

In [9]:
pd.read_sql('''
SELECT
  product_name,
  category,
  SUM(qty_ordered) AS total_ordered
FROM 
  pop_profits_sale
WHERE 
  1 = 1 
  AND year = '2016' 
  AND category = "Furniture"
GROUP BY
  1
ORDER BY
  3 DESC
LIMIT
  1
''', conn)

,product_name,category,total_ordered
0,"Global Push Button Manager's Chair, Indigo",Furniture,22


## 10. Which customer got the most discount in the data? (in total amount)

-- customer_ID | customer_name | total_discounts

In [6]:
pd.read_sql('''
SELECT 
  c.customer_id AS customer_ID, 
  c.customer_name AS customer_name, 
  SUM(od.order_sales - od.order_profits) AS total_discounts 
FROM 
  customers AS c 
  JOIN orders AS o ON c.customer_id = o.customer_id 
  JOIN order_details AS od ON o.order_id = od.order_id 
GROUP BY 
  1 
ORDER BY 
  3 DESC 
LIMIT 
  1
''', conn)

,customer_ID,customer_name,total_discounts
0,687,Sean Miller,27023


## 11. How widely did monthly profits vary in 2018?

***pop_profits_sale VIEW:***

*order_id | state | city | product_name | category | qty_ordered | sales | profits | profit_margin | region | day | month | year*

-- month | monthly_profits | previous_month | profit_diff

Fixing the order by issue: 
    https://stackoverflow.com/questions/16519648/sql-for-ordering-by-number-1-2-3-4-etc-instead-of-1-10-11-12

In [6]:
pd.read_sql('''
WITH t1 AS (
SELECT
    month,
    SUM(profits) AS monthly_profits,
    LAG(SUM(profits), 1,0) OVER (ORDER BY LENGTH(month), month) AS previous_month
FROM pop_profits_sale
WHERE year = '2018'
GROUP BY 
  1
)
SELECT 
  t1.month,
  t1.monthly_profits,
  t1.previous_month,
  SUM(t1.monthly_profits - t1.previous_month) AS profit_diff
FROM t1
GROUP BY 1
ORDER BY 
  LENGTH(month), month
''', conn)

,month,monthly_profits,previous_month,profit_diff
0,1,7137,0,7137
1,2,1612,7137,-5525
2,3,14758,1612,13146
3,4,934,14758,-13824
4,5,6342,934,5408
5,6,8226,6342,1884
6,7,6951,8226,-1275
7,8,9034,6951,2083
8,9,10987,9034,1953
9,10,9272,10987,-1715


## 12. Which order was the highest in 2015?

***pop_profits_sale VIEW:***

*order_id | state | city | product_name | category | qty_ordered | sales | profits | profit_margin | region | day | month | year*

-- order_id  |  total_sales_2015

In [7]:
pd.read_sql('''
SELECT 
  order_id, 
  SUM(sales) AS total_sales_2015 
FROM 
    pop_profits_sale
WHERE 
  year = '2015'
GROUP BY
  1
ORDER BY
  2 DESC
LIMIT
  1
''', conn)

,order_id,total_sales_2015
0,CA-2015-145317,23660


## 13. What was the rank of each city in the East region in 2015?

***pop_profits_sale VIEW:***

*order_id | state | city | product_name | category | qty_ordered | sales | profits | profit_margin | region | day | month | year*

-- city | num_products_sold | rank

In [11]:
#ALL East region cities ranked by total orders
pd.read_sql('''

SELECT 
  city,
  SUM(qty_ordered) AS num_products_sold,
  RANK()OVER (ORDER BY SUM(qty_ordered) DESC) AS rank 
FROM 
  pop_profits_sale 
WHERE 
  1 = 1 
  AND region = "East" 
  AND year = '2015' 
GROUP BY 
  1 
''', conn)

,city,num_products_sold,rank
0,New York City,1708,1
1,Philadelphia,403,2
2,Columbus,167,3
3,Newark,64,4
4,Fairfield,53,5
5,Long Beach,44,6
6,Lakewood,38,7
7,Lancaster,36,8
8,Lawrence,31,9
9,Dover,30,10


#### 14. Join all DB tables into one dataset that includes all unique columns and download it as a csv file. In the second part of the project, you're gonna work with this one table.

In [23]:
#read each csv file into a dataframe
df_sample1 = pd.read_csv("customers.csv")
df_sample1.head()

,customer_id,customer_name,customer_segment
0,1,Aaron Bergman,Consumer
1,2,Aaron Hawkins,Corporate
2,3,Aaron Smayling,Corporate
3,4,Adam Bellavance,Home Office
4,5,Adam Hart,Corporate


In [24]:
df_sample2 = pd.read_csv("orders.csv")
df_sample2.head()

,order_id,customer_id,order_date,shipping_city,shipping_state,shipping_region,shipping_country,shipping_postal_code,shipping_date,shipping_mode
0,CA-2015-100004,91,9/6/2015,New York City,New York,East,United States,10011.0,9/6/2015,Same Day
1,CA-2015-100006,224,9/7/2015,New York City,New York,East,United States,10024.0,9/13/2015,Standard Class
2,CA-2015-100032,62,9/7/2015,New York City,New York,East,United States,10025.0,9/13/2015,Standard Class
3,CA-2015-100051,311,9/6/2015,New York City,New York,East,United States,10011.0,9/12/2015,Standard Class
4,CA-2015-100090,246,7/8/2015,San Francisco,California,West,United States,94122.0,7/12/2015,Standard Class


In [25]:
df_sample3 = pd.read_csv("order_details.csv")
df_sample3.head()

,order_details_id,order_id,product_id,quantity,order_discount,order_profits,order_profit_ratio,order_sales
0,1,CA-2015-100004,122,62,0.1,327,0.39,837
1,2,CA-2015-100004,764,18,0.2,565,0.26,2174
2,3,CA-2015-100006,161,3,0.0,110,0.29,378
3,4,CA-2015-100032,122,53,0.1,272,0.38,715
4,5,CA-2015-100032,1342,23,0.1,96,0.31,311


In [26]:
df_sample4 = pd.read_csv("product.csv")
df_sample4.head()

,product_id,product_name,product_category,product_subcategory,product_manufacturer
0,1,"""While you Were Out"" Message Book, One Form pe...",Office Supplies,Paper,Other
1,2,"#10 Gummed Flap White Envelopes, 100/Box",Office Supplies,Envelopes,Other
2,3,#10 Self-Seal White Envelopes,Office Supplies,Envelopes,Other
3,4,"#10 White Business Envelopes,4 1/8 x 9 1/2",Office Supplies,Envelopes,Other
4,5,"#10- 4 1/8"" x 9 1/2"" Recycled Envelopes",Office Supplies,Envelopes,Other


In [45]:
#merge dataframes
df_master = df_sample1.merge(df_sample2,
                   on = 'customer_id', 
                   how = 'outer')
# check if merged correctly 
#df_master

In [46]:
df_master = df_master.merge(df_sample3,
                   on = 'order_id', 
                   how = 'outer')
# check if merged correctly 
#df_master

In [47]:
df_master = df_master.merge(df_sample4,
                   on = 'product_id', 
                   how = 'outer')
# check if merged correctly 
df_master.head()

,customer_id,customer_name,customer_segment,order_id,order_date,shipping_city,shipping_state,shipping_region,shipping_country,shipping_postal_code,...,product_id,quantity,order_discount,order_profits,order_profit_ratio,order_sales,product_name,product_category,product_subcategory,product_manufacturer
0,1,Aaron Bergman,Consumer,CA-2015-152905,2/18/2015,Arlington,Texas,Central,United States,76017.0,...,117.0,2.0,0.2,-3.0,-0.20,13.0,Akro Stacking Bins,Office Supplies,Storage,Akro
1,6,Adam Shillingsburg,Consumer,CA-2017-104311,5/2/2017,Irving,Texas,Central,United States,75061.0,...,117.0,3.0,0.2,-4.0,-0.20,19.0,Akro Stacking Bins,Office Supplies,Storage,Akro
2,7,Adrian Barton,Consumer,CA-2018-153822,9/19/2018,Phoenix,Arizona,West,United States,85023.0,...,117.0,2.0,0.2,-3.0,-0.20,13.0,Akro Stacking Bins,Office Supplies,Storage,Akro
3,460,Lena Radford,Consumer,CA-2017-126732,9/2/2017,Los Angeles,California,West,United States,90036.0,...,117.0,3.0,0.0,1.0,0.04,24.0,Akro Stacking Bins,Office Supplies,Storage,Akro
4,520,Meg Tillman,Consumer,CA-2017-104157,7/25/2017,Jackson,Mississippi,South,United States,39212.0,...,117.0,1.0,0.0,0.0,0.04,8.0,Akro Stacking Bins,Office Supplies,Storage,Akro


In [ ]:
#Export combined dataframe as a csv file
df_master.to_csv('Combined_capfiles.csv')